## Part 2: Vice Presidential and Presidential Debate Analysis
* Debates are a tool for the candidates to appeal to an audience deciding who to vote for in the election. 
    * Viewers of these debates might already have a candidate in mind, but the audience isn't directed towards only one side, like the conventions. 
* This analysis will include a classifier, which will try to predict if a spoken line is republican or democratic.
    * Goal: find key words used by each candidate to further their arguments. Also, find accuracy of the classifier for my hypothesis. 
* I will also be making a model based off of token length to see if speaking a lot is a persuasion choice made by the candidates. 

In [1]:
import pandas as pd

In [2]:
presdebate = pd.read_csv("/Users/emmatarcson/Documents/data_science/RhetoricalFactor-analysis/data/presdebatewelker.csv")
presdebate = presdebate[['Aff','transcript']]
presdebate

,Aff,transcript
0,None,"Good evening, everyone. Good evening. Thank yo..."
1,R,How are you doing? How are you?
2,None,And I do want to say a very good evening to bo...
3,None,The goal is for you to hear each other and for...
4,None,… during this next stage of the coronavirus cr...
...,...,...
507,None,"All right. Vice President Biden, same question..."
508,D,"I will say, I’m an American President. I repre..."
509,D,"We can grow this economy, we can deal with the..."
510,None,"All right, I want to thank you both for a very..."


In [3]:
vpdebate = pd.read_csv("/Users/emmatarcson/Documents/data_science/RhetoricalFactor-analysis/data/pagedebate2020.csv")
vpdebate = vpdebate[['Aff','transcript']]
vpdebate

,Aff,transcript
0,None,Good evening. From the University of Utah in S...
1,None,"These are tumultuous times, but we can and wil..."
2,D,"Thank you, Susan. Well, the American people ha..."
3,D,"Can you imagine if you knew on January 28th, a..."
4,None,"Thank you, Senator Harris-"
...,...,...
322,D,"And brings me to Joe, Joe Biden. One of the re..."
323,D,Joe has a longstanding reputation of working a...
324,D,"Brecklin, when you think about the future, I d..."
325,None,"Thank you, Senator Harris. Thank you, vice pre..."
